**BASIC CIFAS EXAMPLE WITH CONV**

Imports...

In [1]:
from numpy.random import seed
seed(86148)

from __future__ import print_function

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization as BN
from tensorflow.keras.layers import GaussianNoise as GN
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler as LRS

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
batch_size = 100
num_classes = 10
epochs = 75

In [3]:
#### LOAD AND TRANSFORM
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print(x_train.shape)
print(x_test.shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [4]:
## DEF A BLOCK CONV + BN + GN + MAXPOOL
def CBGN(model,filters,ishape=0):
    
    if (ishape!=0):
        model.add(Conv2D(filters, (3, 3), padding='same', input_shape=ishape))
    else:
        model.add(Conv2D(filters, (3, 3), padding='same'))
  
    model.add(BN())
    model.add(GN(0.2))
    model.add(Activation('relu'))
    
    
    model.add(Conv2D(filters, (3, 3), padding='same'))    
    model.add(BN())
    model.add(GN(0.2))
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
  
  
    return model


In [5]:
## DEF NN TOPOLOGY  
model = Sequential()

model=CBGN(model,32,x_train.shape[1:])
model=CBGN(model,64)
model=CBGN(model,128)
model=CBGN(model,256)
model=CBGN(model,512)

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))

model.add(Dense(num_classes))
model.add(Activation('softmax'))


model.summary()


Metal device set to: Apple M1
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
gaussian_noise (GaussianNois (None, 32, 32, 32)        0         
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
gaussian_noise_1 (Gaussian

2021-12-26 21:33:59.592147: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-26 21:33:59.592232: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
## OPTIM AND COMPILE
opt = SGD(learning_rate=0.1, decay=1e-6)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


In [7]:

# OPTIM AND COMPILE
opt = SGD(learning_rate=0.1, decay=1e-6)

datagen = ImageDataGenerator(
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    zoom_range=0.2,
    rotation_range = 20
)


def scheduler(epoch):
    if epoch < 50:
        return .1
    elif epoch < 80:
        return 0.05
    elif epoch < 100:
        return 0.01
    else:
        return 0.001


filepath_val_acc="./backup/model.best.hdf5"
checkpoint_val_acc = ModelCheckpoint(filepath_val_acc, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint_val_acc]    
    
## TRAINING
history = model.fit(
    datagen.flow(x_train, y_train,batch_size=batch_size),             
    epochs=epochs,
    verbose=1,
    validation_data=(x_test, y_test),
    shuffle=True,
    callbacks=[LRS(scheduler), checkpoint_val_acc]
)

2021-12-26 21:34:00.042744: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-26 21:34:00.042870: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/75


2021-12-26 21:34:00.291510: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


500/500 [==============================] - ETA: 0s - loss: 1.9725 - accuracy: 0.2676

2021-12-26 21:34:57.226392: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


500/500 [==============================] - 60s 119ms/step - loss: 1.9725 - accuracy: 0.2676 - val_loss: 1.7108 - val_accuracy: 0.3590

Epoch 00001: val_accuracy improved from -inf to 0.35900, saving model to ./backup/model.best.hdf5
Epoch 2/75
500/500 [==============================] - 59s 117ms/step - loss: 1.5658 - accuracy: 0.4217 - val_loss: 1.5151 - val_accuracy: 0.4703

Epoch 00002: val_accuracy improved from 0.35900 to 0.47030, saving model to ./backup/model.best.hdf5
Epoch 3/75
500/500 [==============================] - 59s 117ms/step - loss: 1.3571 - accuracy: 0.5073 - val_loss: 1.3418 - val_accuracy: 0.5133

Epoch 00003: val_accuracy improved from 0.47030 to 0.51330, saving model to ./backup/model.best.hdf5
Epoch 4/75
500/500 [==============================] - 59s 118ms/step - loss: 1.2004 - accuracy: 0.5727 - val_loss: 1.3024 - val_accuracy: 0.5748

Epoch 00004: val_accuracy improved from 0.51330 to 0.57480, saving model to ./backup/model.best.hdf5
Epoch 5/75
500/500 [======

500/500 [==============================] - 59s 117ms/step - loss: 0.4918 - accuracy: 0.8279 - val_loss: 0.4570 - val_accuracy: 0.8465

Epoch 00038: val_accuracy improved from 0.84420 to 0.84650, saving model to ./backup/model.best.hdf5
Epoch 39/75
500/500 [==============================] - 59s 118ms/step - loss: 0.4805 - accuracy: 0.8342 - val_loss: 0.4958 - val_accuracy: 0.8448

Epoch 00039: val_accuracy did not improve from 0.84650
Epoch 40/75
500/500 [==============================] - 59s 117ms/step - loss: 0.4772 - accuracy: 0.8332 - val_loss: 0.5419 - val_accuracy: 0.8271

Epoch 00040: val_accuracy did not improve from 0.84650
Epoch 41/75
500/500 [==============================] - 59s 117ms/step - loss: 0.4675 - accuracy: 0.8382 - val_loss: 0.5220 - val_accuracy: 0.8379

Epoch 00041: val_accuracy did not improve from 0.84650
Epoch 42/75
500/500 [==============================] - 59s 117ms/step - loss: 0.4693 - accuracy: 0.8384 - val_loss: 0.4434 - val_accuracy: 0.8522

Epoch 00042

In [8]:
## TEST
score = model.evaluate(x_test, y_test, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
print(f"Test error: {((1-score[1]) * 100):.3f}%")

313/313 [==============================] - 3s 9ms/step - loss: 0.3753 - accuracy: 0.8810
Test loss: 0.37526604533195496
Test accuracy: 0.8810000419616699
Test error: 11.900%
